In [1]:
import pandas as pd
import numpy as np

In [2]:
popularity_data = pd.read_csv(r"C:\Users\guyon\OneDrive\Desktop\igdb_data\igdb_data\igdb_popularity_primitives_20250925_1153.csv")
time_to_beat = pd.read_csv(r"C:\Users\guyon\OneDrive\Desktop\igdb_data\igdb_data\igdb_game_time_to_beats_20250925_1112.csv")
games = pd.read_csv(r"C:\Users\guyon\OneDrive\Desktop\igdb_data\igdb_data\igdb_games_20250925_1105.csv")

In [3]:
# Print number of rows in games
print(len(games))

# Count nulls in each column
null_counts = games.isnull().sum()

# Filter only columns with null values
null_df = null_counts[null_counts > 0].reset_index()
null_df.columns = ["column", "null_count"]

# Print the results
print(null_df)

# print(null_df.iloc[[23, 24, 25, 26, 27]])


10000
                     column  null_count
0               age_ratings        2200
1         aggregated_rating        5053
2   aggregated_rating_count        5053
3         alternative_names        3985
4                  artworks        6380
5                   bundles        7666
6                     cover          63
7            external_games          76
8        first_release_date         131
9              game_engines        7856
10               game_modes         290
11                   genres         145
12       involved_companies         257
13                 keywords         790
14        multiplayer_modes        8162
15                platforms          64
16      player_perspectives        1441
17                   rating        2931
18             rating_count        2931
19            release_dates          80
20              screenshots        1352
21            similar_games          54
22                storyline        7575
23                  summary       

In [5]:
# list of columns to drop in games table that have null values and are not needed for analysis
cols_to_drop = ['age_ratings', 'alternative_names', 'artworks', 'bundles', 'cover', 'external_games', 
                'first_release_date', 'game_engines', 'game_modes', 'involved_companies', 'keywords',
                'multiplayer_modes', 'screenshots', 'similar_games', 'storyline', 'summary', 'tags',
                'themes', 'videos', 'websites', 'language_supports', 'collections']
games = games.drop(columns=cols_to_drop)
print(games.columns)

Index(['id', 'aggregated_rating', 'aggregated_rating_count', 'created_at',
       'genres', 'name', 'platforms', 'player_perspectives', 'rating',
       'rating_count', 'release_dates', 'slug', 'total_rating',
       'total_rating_count', 'updated_at', 'url', 'checksum', 'game_type'],
      dtype='object')


In [ ]:
# Fill nulls in string columns with 'Unknown'
for col in ['genre', 'platforms', 'player_perspectives']:
    if col in games.columns:
        games[col] = games[col].fillna('Unknown')

# Fill nulls with average for numeric columns
for col in ['aggregated_rating', 'aggregated_rating_count', 'rating', 'rating_count', 'total_rating', 'total_rating_count']:
    if col in games.columns:
        avg_value = games[col].mean()
        games[col] = games[col].fillna(avg_value)

# Convert 'aggregated_rating', 'rating', and 'total_rating' to floats and 1 decimal place
for col in ['aggregated_rating', 'rating', 'total_rating']:
    if col in games.columns:
        games[col] = games[col].astype(float).round(1)

# Fill nulls in 'release_dates' with median date:
# if 'release_dates' in games.columns:
   #  median_date = games['release_dates'].median()
   #  games['release_dates'] = games['release_dates'].fillna(median_date)

In [ ]:
# Print number of rows in time_to_beat
print(len(time_to_beat))

# Count nulls in each column
null_counts = time_to_beat.isnull().sum()

# Filter only columns with null values
null_df = null_counts[null_counts > 0].reset_index()
null_df.columns = ["column", "null_count"]

# Print the results
print(null_df)

6092
       column  null_count
0     hastily        3008
1    normally        1288
2  completely        2623


In [ ]:
# Fill nulls with average for numeric columns
for col in ['hastily', 'normally', 'completely']:
    if col in time_to_beats.columns:
        avg_value = time_to_beat[col].mean()
        time_to_beat[col] = time_to_beat[col].fillna(avg_value)

222990
Empty DataFrame
Columns: [column, null_count]
Index: []
